In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:22pt;padding:5px;}
table.dataframe{font-size:22px;}
</style>
"""))

<b><font size="7" color="red">ch11. 데이터프레임과 시리즈(Pandas)</font></b>
# 9절 데이터그룹화 및 집계

In [3]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
print(iris.shape)
iris.iloc[[0,49,50,99,100,149]]

(150, 5)


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
99,5.7,2.8,4.1,1.3,versicolor
100,6.3,3.3,6.0,2.5,virginica
149,5.9,3.0,5.1,1.8,virginica


# 9.1 groupby
cf. pivot_table

In [11]:
# 1) 그룹화방법1
#    df.groupby(by=[그룹핑할 열])[[함수적용해서출력할열들]].기초통계함수()
    # [[함수적용해서출력할열들]] 생략시 모든 열에 적용됨
iris_grouped = iris.groupby(by='Species')
iris_grouped = iris.groupby(by=iris.Species)
iris_grouped_sum = iris_grouped.sum()
iris_grouped_sum # iris.groupby(by='Species').sum()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
versicolor,296.8,138.5,213.0,66.3
virginica,329.4,148.7,277.6,101.3


In [12]:
# setosa와 virginica의 종별 합
iris_grouped_sum.loc[['setosa', 'virginica']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,250.3,171.4,73.1,12.3
virginica,329.4,148.7,277.6,101.3


In [17]:
iris_grouped_sum.reset_index(inplace=True)
iris_grouped_sum

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [20]:
# setosa와 virginica의 종별 합
iris_grouped_sum[(iris_grouped_sum['Species']=='setosa' ) | 
                 (iris_grouped_sum['Species']=='virginica' )]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
2,virginica,329.4,148.7,277.6,101.3


In [21]:
iris_grouped_sum[iris_grouped_sum['Species'].isin(['setosa','virginica'])]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
2,virginica,329.4,148.7,277.6,101.3


In [23]:
# 1) 그룹화방법1-1 : 그룹핑할 때 index를 컬럼으로 편입
#    df.groupby(by=[그룹핑할 열], as_index=False)[[함수적용해서출력할열들]].기초통계함수()
    # [[함수적용해서출력할열들]] 생략시 모든 열에 적용됨
iris.groupby('Species', as_index=False).sum()

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [ ]:
# 1) 그룹화방법1 : 그룹핑할 때 index를 컬럼으로 편입
#    df.groupby(by=[그룹핑할 열])[[함수적용해서출력할열들]].기초통계함수()
    # [[함수적용해서출력할열들]] 생략시 모든 열에 적용됨
# 2) 그룹화방법2 
#    df.pivot_table(index='그룹화할 열',
                #   values=[함수적용해서출력할열들],
                #   aggfunc='기초통계함수')

In [10]:
iris.pivot_table(index='Species',
                 # values=['Sepal.Length'],
                aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


## 다중열 그룹화

In [15]:
# iris를 다중열로 그룹화하기 위해 데이터 추가(num필드)
# 0-25개, 1-25개, 2-25개, 3-25개, 4-25개, 5-5개
import numpy as np
np.array([[i]*25 for i in range(6)]).ravel()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [18]:
iris['num'] = np.array([[i]*25 for i in range(6)]).ravel()
iris.iloc[::10]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
10,5.4,3.7,1.5,0.2,setosa,0
20,5.4,3.4,1.7,0.2,setosa,0
30,4.8,3.1,1.6,0.2,setosa,1
40,5.0,3.5,1.3,0.3,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
60,5.0,2.0,3.5,1.0,versicolor,2
70,5.9,3.2,4.8,1.8,versicolor,2
80,5.5,2.4,3.8,1.1,versicolor,3
90,5.5,2.6,4.4,1.2,versicolor,3
